In [69]:
# Mission 1. 여러 이메일이 섞여 있는 문자열에서 도메인(@ 뒤 문자열)만 리스트로 만들어서 출력해주세요.

import re

text = "문의: help@naver.com, admin@gmail.com, sales@myshop.co.kr"

regex = re.compile("\@+[a-z]+\.+[a-z]+")

print(regex.findall(text))

['@naver.com', '@gmail.com', '@myshop.co']


In [27]:
# Mission 2. 아래 리뷰 문장을 한 줄 문자열로 정리해주세요.
text = """
이 제품은 정말 좋아요!
배송도 빠르고
    포장도 깔끔해요.
다음에 또 구매할게요 :)
"""

def clean_one_line(text) :
    if not text :
        return ""
    text = text.replace("\n", " ")
    return " ".join(text.split())
    
print(clean_one_line(text))

이 제품은 정말 좋아요! 배송도 빠르고 포장도 깔끔해요. 다음에 또 구매할게요 :)


In [9]:
import os
import requests
import pandas as pd

API_KEY = os.getenv("TMDB_API_KEY", "f23e4437a60cdd6233afc765d5d70921")
NOW_LANG = "ko-KR"
PAGE = 1
GENRE_LANG = "ko"
POSTER_BASE = "https://image.tmdb.org/t/p/w500"
BACKDROP_BASE = "https://image.tmdb.org/t/p/w780"

def genre_ids_to_names(ids, genre_map: dict) -> str :
    if not isinstance(ids, list) :
        return ""
    names = [genre_map.get(i, str(i)) for i in ids]
    return "|".join(names)

def clean_one_line(text: str) -> str :
    if not text :
        return ""
    text = text.replace("\n", " ")
    return " ".join(text.split())

def fetch_genre_map(api_key: str, language: str ="ko") -> dict :
    url = "https://api.themoviedb.org/3/genre/movie/list"
    params = {"api_key": api_key, "language": language}
    res = requests.get(url, params=params, timeout=20)
    res.raise_for_status()

    genres = res.json().get("genres", [])
    return {g["id"] : g["name"] for g in genres}

genre_map = fetch_genre_map(API_KEY, GENRE_LANG)

url = "https://api.themoviedb.org/3/movie/now_playing"
params = {"api_key": API_KEY, "language": NOW_LANG, "page": PAGE}

res = requests.get(url, params=params, timeout=20)
res.raise_for_status()

data = res.json()

rows = []

for m in data.get("results", []) :
    poster_path = m.get("poster_path")
    backdrop_path = m.get("backdrop_path")

    poster_url = f"{POSTER_BASE}{poster_path}" if poster_path else ""
    backdrop_url = f"{BACKDROP_BASE}{backdrop_path}" if backdrop_path else ""

    genre_ids = m.get("genre_ids", [])

    rows.append({
        "id": m.get("id"),
        "title": m.get("title"),
        "original_title": m.get("original_title"),
        "overview": m.get("overview"),
        "genre_ids": ",".join(map(str, genre_ids)) if isinstance(genre_ids, list) else "",
        "genre_names": genre_ids_to_names(genre_ids, genre_map),
        "adult": m.get("adult"),
        "original_language": m.get("original_language"),
        "vote_average": m.get("vote_average"),
        "vote_count": m.get("vote_count"),
        "poster_url": poster_url,
        "backdrop_path": backdrop_url   
    })

df = pd.DataFrame(rows, columns=[
    "id",
    "title",
    "original_title",
    "overview",
    "genre_ids",
    "genre_names",
    "adult",
    "original_language",
    "vote_average",
    "vote_count",
    "poster_url",
    "backdrop_path"
])

out_path = "tmdb_now_playing_prct01.csv"
df.to_csv(out_path, index=False, encoding="utf-8-sig")

print(f"✅ 저장완료 : {out_path}(rows={len(df)})")

df.head()

✅ 저장완료 : tmdb_now_playing_prct01.csv(rows=20)


,id,title,original_title,overview,genre_ids,genre_names,adult,original_language,vote_average,vote_count,poster_url,backdrop_path
0,1306368,더 립,The Rip,허름한 은닉처에서 수백만 달러의 현금을 발견한 마이애미 경찰. 신뢰에 금이 가기 시...,"28,53,80",액션|스릴러|범죄,False,en,7.033,842,https://image.tmdb.org/t/p/w500/o0d6Us9VWOW0nH...,https://image.tmdb.org/t/p/w780/3F2EXWF1thX0Bd...
1,1043197,더스트 바니,Dust Bunny,“어른도 아이도 함께하는 몬스터 사냥 이야기” 열 살 소녀 오로라는 침대 밑에 숨...,"28,14,53",액션|판타지|스릴러,False,en,6.564,128,https://image.tmdb.org/t/p/w500/vobigFZFvbYPf6...,https://image.tmdb.org/t/p/w780/AecGG1XVCmkk7f...
2,83533,아바타: 불과 재,Avatar: Fire and Ash,"인간들과의 전쟁으로 첫째 아들 ‘네테이얌’을 잃은 후, ‘제이크’와 ‘네이티리’는 ...","878,12,14",SF|모험|판타지,False,en,7.308,1631,https://image.tmdb.org/t/p/w500/l18o0AK18KS118...,https://image.tmdb.org/t/p/w780/3Dqievkc7krcTt...
3,991494,스폰지밥 무비: 네모바지를 찾아서,The SpongeBob Movie: Search for SquarePants,비키니시티에 사는 스폰지밥은 무시무시한 롤러코스터를 탈 수 있는 용감한 ‘빅 가이’...,"16,10751,35,12,14",애니메이션|가족|코미디|모험|판타지,False,en,6.526,136,https://image.tmdb.org/t/p/w500/2QCGC4toQXYuIs...,https://image.tmdb.org/t/p/w780/gbjK8p5S1aLXWC...
4,1368166,하우스메이드,The Housemaid,과거를 숨긴 채 완벽한 저택의 가정부로 들어간 밀리가 이상한 행동을 보이는 아내 니...,"9648,53",미스터리|스릴러,False,en,7.102,542,https://image.tmdb.org/t/p/w500/fUDX16A4fJXmAu...,https://image.tmdb.org/t/p/w780/sK3z0Naed3H1Wu...
